In [4]:
from supabase import create_client
import pandas as pd
import os
from dotenv import load_dotenv

# -----------------------------------------
# 1️⃣ Load .env and connect to Supabase
# -----------------------------------------
load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("❌ Missing SUPABASE_URL or SUPABASE_KEY in .env")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

print("🔗 Connected to Supabase!")

# -----------------------------------------
# 2️⃣ Fetch Titanic Data
# -----------------------------------------
response = supabase.table("titanic_data").select("*").execute()
df = pd.DataFrame(response.data)

print("\n📌 DATA LOADED SUCCESSFULLY\n")
print(df.head())

# -----------------------------------------
# 3️⃣ Basic Dataset Info
# -----------------------------------------
print("\n====== DATA INFO ======\n")
print(df.info())

print("\n====== SHAPE ======\n")
print(df.shape)

print("\n====== NULL VALUES ======\n")
print(df.isnull().sum())

print("\n====== DESCRIPTIVE STATS ======\n")
print(df.describe(include="all"))

print("\n====== COLUMNS ======\n")
print(df.columns)

# -----------------------------------------
# 4️⃣ Handling Missing Values
# -----------------------------------------
print("\n===== FILLING MISSING VALUES =====\n")

if "age" in df.columns:
    df["age"].fillna(df["age"].median(), inplace=True)

if "embarked" in df.columns:
    df["embarked"].fillna(df["embarked"].mode()[0], inplace=True)

if "fare" in df.columns:
    df["fare"].fillna(df["fare"].median(), inplace=True)

print("Missing values after filling:")
print(df.isnull().sum())

# -----------------------------------------
# 5️⃣ Detect Duplicates
# -----------------------------------------
print("\n===== DUPLICATES =====\n")
print(f"Duplicate rows: {df.duplicated().sum()}")

# -----------------------------------------
# 6️⃣ Survival Analysis (updated with correct column names)
# -----------------------------------------
if "survived" in df.columns:

    print("\n===== SURVIVAL BY SEX =====\n")
    print(df.groupby("sex")["survived"].mean())

    print("\n===== SURVIVAL BY PCLASS =====\n")
    print(df.groupby("pclass")["survived"].mean())

    # Age Group Creation
    df["age_group"] = pd.cut(
        df["age"],
        bins=[0, 12, 18, 35, 60, 100],
        labels=["Child", "Teen", "Adult", "Middle Age", "Senior"]
    )

    print("\n===== SURVIVAL BY AGE GROUP =====\n")
    print(df.groupby("age_group")["survived"].mean())

# -----------------------------------------
# 7️⃣ Correlation Matrix
# -----------------------------------------
print("\n===== CORRELATION MATRIX =====\n")
numeric_df = df.select_dtypes(include="number")
print(numeric_df.corr())

# -----------------------------------------
# 8️⃣ Categorical Counts
# -----------------------------------------
if "sex" in df.columns:
    print("\n===== SEX COUNTS =====\n")
    print(df["sex"].value_counts())

if "embarked" in df.columns:
    print("\n===== EMBARKED COUNTS =====\n")
    print(df["embarked"].value_counts())

# -----------------------------------------
# 9️⃣ Feature Engineering (correct names)
# -----------------------------------------
if "sibsp" in df.columns and "parch" in df.columns:
    df["family_size_calc"] = df["sibsp"] + df["parch"] + 1
    df["is_alone_calc"] = (df["family_size_calc"] == 1).astype(int)

print("\n===== NEW FEATURES ADDED =====\n")
print(df[["family_size_calc", "is_alone_calc"]].head())

print("\n🎉 FULL ANALYSIS COMPLETED SUCCESSFULLY!")


🔗 Connected to Supabase!

📌 DATA LOADED SUCCESSFULLY

   id  survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0   1         0       3    male  22.0      1      0   7.2500        S  Third   
1   2         1       1  female  38.0      1      0  71.2833        C  First   
2   3         1       3  female  26.0      0      0   7.9250        S  Third   
3   4         1       1  female  35.0      1      0  53.1000        S  First   
4   5         0       3    male  35.0      0      0   8.0500        S  Third   

     who     deck  embark_town  alone  family_size  is_alone  title  
0    man  Unknown  Southampton  False            2     False    Man  
1  woman        C    Cherbourg  False            2     False  Woman  
2  woman  Unknown  Southampton   True            1      True  Woman  
3  woman        C  Southampton  False            2     False  Woman  
4    man  Unknown  Southampton   True            1      True    Man  

====== DATA INFO ======

<class 'pandas.core

C:\Users\ashri\AppData\Local\Temp\ipykernel_103920\3159596783.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["age"].fillna(df["age"].median(), inplace=True)
C:\Users\ashri\AppData\Local\Temp\ipykernel_103920\3159596783.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [10]:
import pandas as pd
import numpy as np
import os

def transform_telco(input_path, output_path, display_rows=10):
    # --------------------------------------------------
    # 0️⃣ Check if input file exists
    # --------------------------------------------------
    if not os.path.exists(input_path):
        raise FileNotFoundError(
            f"❌ Input CSV not found at: {input_path}\n"
            f"ℹ️  Please download it from Kaggle and place it here."
        )
    
    print(f"🔍 Loading dataset from: {input_path}")
    df = pd.read_csv(input_path)
    print(f"📌 Initial shape: {df.shape}")

    # ============================================================
    # 1️⃣ CLEANING TASKS
    # ============================================================
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].replace(" ", np.nan), errors="coerce")
    for col in ["tenure", "MonthlyCharges", "TotalCharges"]:
        df[col].fillna(df[col].median(), inplace=True)
    for col in df.select_dtypes(include=["object"]).columns:
        df[col].fillna("Unknown", inplace=True)

    # ============================================================
    # 2️⃣ FEATURE ENGINEERING
    # ============================================================
    df["tenure_group"] = pd.cut(
        df["tenure"],
        bins=[0, 12, 36, 60, 1000],
        labels=["New", "Regular", "Loyal", "Champion"],
        right=True
    )
    df["monthly_charge_segment"] = pd.cut(
        df["MonthlyCharges"],
        bins=[0, 30, 70, 500],
        labels=["Low", "Medium", "High"]
    )
    df["has_internet_service"] = df["InternetService"].map({
        "DSL": 1,
        "Fiber optic": 1,
        "No": 0
    }).fillna(0).astype(int)
    df["is_multi_line_user"] = df["MultipleLines"].apply(lambda x: 1 if x == "Yes" else 0)
    df["contract_type_code"] = df["Contract"].map({
        "Month-to-month": 0,
        "One year": 1,
        "Two year": 2
    }).fillna(0).astype(int)

    # ============================================================
    # 3️⃣ DROP UNNEEDED COLUMNS
    # ============================================================
    df.drop(columns=["customerID", "gender"], inplace=True, errors="ignore")

    # ============================================================
    # 4️⃣ EXPORT TRANSFORMED DATA
    # ============================================================
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"🎉 Transformation complete! Saved to: {output_path}")
    print(f"📊 Final shape: {df.shape}")

    # ============================================================
    # 5️⃣ DISPLAY DATAFRAME INFORMATION
    # ============================================================
    print("\n🖥 Showing the first few rows of the transformed dataset:")
    print(df.head(display_rows))
    
    print("\nℹ️ Dataset Info:")
    print(df.info())

    print("\n📊 Statistical Summary of Numeric Columns:")
    print(df.describe())

    print("\n🗂 Column Names:")
    print(df.columns.tolist())

    return df


# ============================================================
# RUN SCRIPT
# ============================================================
if __name__ == "__main__":
    input_file = "data/raw/Telco-Customer-Churn.csv"      # <-- update path if needed
    output_file = "data/staged/telco_transformed.csv"
    
    try:
        transform_telco(input_file, output_file)
    except FileNotFoundError as e:
        print(e)


🔍 Loading dataset from: data/raw/Telco-Customer-Churn.csv
📌 Initial shape: (7043, 21)
🎉 Transformation complete! Saved to: data/staged/telco_transformed.csv
📊 Final shape: (7043, 24)

🖥 Showing the first few rows of the transformed dataset:
   SeniorCitizen Partner Dependents  tenure PhoneService     MultipleLines  \
0              0     Yes         No       1           No  No phone service   
1              0      No         No      34          Yes                No   
2              0      No         No       2          Yes                No   
3              0      No         No      45           No  No phone service   
4              0      No         No       2          Yes                No   
5              0      No         No       8          Yes               Yes   
6              0      No        Yes      22          Yes               Yes   
7              0      No         No      10           No  No phone service   
8              0     Yes         No      28          Yes 

C:\Users\ashri\AppData\Local\Temp\ipykernel_103920\922405011.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\ashri\AppData\Local\Temp\ipykernel_103920\922405011.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa